#  文章目录

[WSGI接口](#1)

[使用Web框架](#2)

[使用模板](#3)

[协程](#4)

# WSGI接口
其实Web应用的本质就是：
- 浏览器发送一个HTTP请求
- 服务器收到请求，生成一个HTML文档
- 服务器把HTML文档作为HTTP响应的Body发送给浏览器
- 浏览器收到HTTP响应，从HTTP Body取出HTML文档并显示

最简单的Web应用就是先把HTML用文件保存好，用一个现成的HTTP服务器软件，接收用户请求，从文件中读取HTML，返回。Apache、Nginx、Lighttpd等这些常见的静态服务器就是干这事的

如果要动态生成HTML，就需要把上述步骤来自己实现。正确的做法事底层代码由专门的服务器软件实现，我们用Python专注于生成HTML文档。因为我们不希望接触到TCP连接、HTTP原始请求和响应格式，所以需要一个同意的接口，让我们专心用Python编写Web业务。

这个接口就是WSGI： Web Server Gateway Interface

WSGI接口定义非常简单，它只要求Web开发者实现一个函数，就可以响应HTTP请求。

In [1]:
#web版本的Hello world
def application(environ, start_response):
    start_response('200 OK',[('Content-Type', 'text/html')])
    return [b'<h1>Hello World!</h1>']

上面的application函数就是符合WSGI标准的一个HTTP处理函数，它接收两个参数：
- envison：一个包含所有HTTP请求信息的dict对象
- start_response: 一个发送HTTP响应的函数
在application函数中，调用：

start_response('200 OK', [('Content-Type', 'text/html')])

就发送了HTTP响应的Header，注意Header只能发送一次，也就是只能调用一次start_response()函数。start_response()函数接收两个参数，一个事HTTP响应码，一个事一组list表示HTTP Header，每个Header用一个包含两个str的tuple表示

通常情况下，都应该把Content-Type头发送给浏览器，其他很多常用的HTTP Header也应该发送



有了WSGI，我们关心的就是如何从environ这个dict对象拿到HTTP请求信息，然后构造HTML，童工start_response()发送Header，最后返回Body

整个application()函数本身没有设计到任何解析HTTP的部分，也就是说底层代码不需要我们自己编写，我们只负责在更高层次上考虑如何响应请求就行l

application()函数必须由WSGI服务器来调用。由很多符合WSGI规范的服务器，我们可以挑选一个来用。但是现在，我们只想尽快测试一下我们编写的application()函数真的可以把HTML输出到浏览器，所以要赶紧找一个最近的WSGI服务器，把我们的Web应用程序跑起来。

好消息事Python内置了一个WSGI服务器，这个模块叫wsgiref，它是用纯Python编写的WSGI服务器的参考实现。所谓参考实现是指实现完全符合WSGI标准，但是不考虑任何运行效率，仅供开发和测试使用。

# 使用Web框架
<a id='2'></a>

了解了WSGI框架，我们发现就是一个Web App，就是写一个WSGI的处理函数，针对每个HTTP请求进行响应。

但是如何处理HTTP请求不是问题，问题是如何处理100哥不同的URL。

每个URL可以对应GET和POST请求，当然还有PUT、DELETE等请求，但是我们通常只考虑最常见的GET和POST请求。

一个最简单的想法是从environ变量中取出HTTP请求的星系，然后逐个判断

但是这么写下去代码肯定没办法维护了，因为WSGI提供的接口虽然比HTTP接口高级不少，但是和Web App处理逻辑相比，还是比较低级，我们需要在WSGI接口上能进一步抽象，让我们专注于用一个函数处理一个URL,至于URL到函数的映射，就交给Web框架来做。

由于用Python开发一个Web框架十分容易，所以Python有上百个开源的Web框架。这里选择一个比较流行的Flask来使用。

写一个app.py处理3个URL，分别是
- GET /：首页，返回Home；
- GET/ signin:登录页，显示登录表单；
- POST/signin：处理登录表单，显示登录结果

注意：同一个URL/signin分别有GET和POST两种请求，映射到两个处理函数中。



#  使用模板
<a id='3'></a>

把上次直接输出字符串作为HTML的例子用MVC改写一下

# 协程
<a id='4'></a>

协程，又称微线程，英文名Coroutine

子程序，或者叫函数，在所有语言中都是层级调用，比如A调用B，B在执行过程中又调用了C，C执行完毕返回，B执行完毕返回，最后是A执行完毕

所以子程序调用是童工栈实现的，一个线程就是执行一个子程序。

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同

协程看上去也是子程序，但执行过程中在子程序内部可终端，然后转而执行别的子程序，在适当的时候再返回来接着执行。

看起来有点像多线程，相比多线程，最大的优势就是协程极高的执行效率。因为子程序切换不是线程切换，而是程序自身控制，因此没有线程切换的开销，和多线程比，线程数量越多，协程的性能优势就越明显。

第二大优势就是不需要多线程的锁机制，因为只有一个线程，也不存在同事写变量冲突，在协程中控制共享资源不加锁，只需要判断状态就好了，所以执行效率比多线程高很多

因为协程是一个线程执行，怎么去利用多核CPU？
最简单的办法是多进程+协程，既充分利用多核，又充分发挥协程的高效率，可获得极高的性能。

Python对协程的支持还非常有限，用在generator中的yield可以一定程度伤实现协程。虽然支持不完全，但是已经可以发挥相当大的为力了。

用协程来做生产者消费者模型：

In [1]:
import time

In [2]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' %n)
        time.sleep(1)
        r = '200 OK'
        
def produce(c):
    c.next()
    n = 0
    while n<5:
        n = n+1
        print('[PRODUCER] Producing %s...' %n)
        r = c.send(n)
        print('[PRODUCER] Cousumer return: %s...' %r)
    c.close()

if __name__ == '__main__':
    c = consumer()
    produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Cousumer return: 200 OK...
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Cousumer return: 200 OK...
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Cousumer return: 200 OK...
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Cousumer return: 200 OK...
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Cousumer return: 200 OK...


注意到consumer函数是一个generator，把一个consumer传入produce后：
- 首先调用c.next()启动生成器
- 然后，一旦生产了东西，通过c.send(n)切换到consumer执行
- consumer通过yield拿到消息，处理，又通过yield把结果传回
- produce拿到consumer处理的结果，继续生产下一条消息
- produce决定不生产了，通过c.close()关闭consumer，整个过程结束

### gevent

Python通过yield提供了对协程的基本支持，但是不完全，第三方的gevent为Python提供了比较完善的协程支持

gevent是第三方库，通过greenlet实现协程，其基本思想：

当一个greenlet遇到IO操作时，比如访问网络，就自动切换到其他的greenlet，等到IO操作完成，再在适当的时候切换回来继续执行。由于IO操作非常耗时，经常使程序处于等待状态，有了gevent为我们自动切换协程，就保证总有greenlet在运行，而不是等待IO

由于切换IO操作时自动完成，所以gevent需要修改Python自带的一些标准库，这一过程在启动时通过monkey patch完成

In [3]:
from gevent import monkey;monkey.patch_socket()

In [4]:
import gevent

In [5]:
def f(n):
    for i in range(n):
        print gevent.getcurrent(), i
        
g1 = gevent.spawn(f, 5)
g2 = gevent.spawn(f, 5)
g3 = gevent.spawn(f, 5)
g1.join()
g2.join()
g3.join()

<Greenlet at 0x61152a8L: f(5)> 0
<Greenlet at 0x61152a8L: f(5)> 1
<Greenlet at 0x61152a8L: f(5)> 2
<Greenlet at 0x61152a8L: f(5)> 3
<Greenlet at 0x61152a8L: f(5)> 4
<Greenlet at 0x61153d8L: f(5)> 0
<Greenlet at 0x61153d8L: f(5)> 1
<Greenlet at 0x61153d8L: f(5)> 2
<Greenlet at 0x61153d8L: f(5)> 3
<Greenlet at 0x61153d8L: f(5)> 4
<Greenlet at 0x6115470L: f(5)> 0
<Greenlet at 0x6115470L: f(5)> 1
<Greenlet at 0x6115470L: f(5)> 2
<Greenlet at 0x6115470L: f(5)> 3
<Greenlet at 0x6115470L: f(5)> 4


可以看到，3个greenlet是依次运行而不是交替运行。
要让greenlet交替运行，可以通过gevent.sleep()交出控制权

In [6]:
def f(n):
    for i in range(n):
        print gevent.getcurrent(), i
        gevent.sleep(0)

实际代码中，一般不会用gevent.sleep去切换协程，而是在执行到IO操作时，gevent自动切换

In [ ]:
from gevent import monkey; monkey.patch_all()
import gevent
import urllib2

def f(url):
    print('GET: %s' %url)
    resp = urllib2.urlopen(url)
    data = resp.read()
    print('%d bytes received from %s' %(len(data), url))
    
gevent.joinall([
    gevent.spawn(f, 'https://www.python.org/'),
    gevent.spawn(f, 'https://www.yahoo.com/'),
    gevent.spawn(f, 'https://github.com/'),
])

The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x61155a0 select pending=0 ref=0>)).History will not be written to the database.
GET: https://www.python.org/
GET: https://www.yahoo.com/
GET: https://github.com/
47430 bytes received from https://www.python.org/
420971 bytes received from https://www.yahoo.com/
93365 bytes received from https://github.com/
